In [1]:
# !pip3 install dotenv

In [2]:
import requests
import json
import numpy as np
import pandas as pd
import threading
from datetime import datetime, timedelta


#API KEYS
from dotenv import load_dotenv
import os

In [3]:
"""
Scope of Work
Project Title: ServiceTitan Timesheet Code Cleaner for Non-Job Events
Script Name: remove_timesheet_codes.py
Owner: Sean Menyhay
Department: Titan Pro Technologies – Software Development

Objective:
The goal of this script is to automate the removal of timesheet codes from non-job-related events 
within ServiceTitan’s scheduling and timesheet system. This ensures data cleanliness, accurate reporting, 
and prevents misallocation of payroll or operational hours to incorrect categories.

Scope:
This script will perform the following:

1. Connect to ServiceTitan API
   - Authenticate using secure API credentials
   - Retrieve timesheet data and associated events within a specified date range or rolling period

2. Identify Non-Job Events
   - Filter out events that are not linked to active jobs, based on criteria such as missing jobId or event type classification

3. Check and Remove Timesheet Codes
   - For each non-job event, inspect for assigned timesheetCode
   - If present, remove or nullify the code by updating the event via the ServiceTitan API

4. Logging and Reporting
   - Generate a log file summarizing:
     - Events processed
     - Codes removed
     - Errors or exceptions (if any)
   - Optional: Email or export a report to CSV for audit purposes

5. Scheduling & Reusability
   - Designed to be run as a scheduled task (e.g., daily or weekly) or on-demand
   - Configurable parameters for date range, tech filters, and logging levels

Out of Scope:
- Manual validation of each timesheet record
- Updating or modifying job-linked timesheet events
- Changing technician schedules or core event data outside timesheet code fields

Dependencies & Requirements:
- Valid ServiceTitan API key with appropriate scopes
- Python 3.9+
- Required libraries: requests, dotenv, pandas (optional, for reporting)
- Access to secure environment for credential storage (.env file or secret manager)
"""

'\nScope of Work\nProject Title: ServiceTitan Timesheet Code Cleaner for Non-Job Events\nScript Name: remove_timesheet_codes.py\nOwner: Sean Menyhay\nDepartment: Titan Pro Technologies – Software Development\n\nObjective:\nThe goal of this script is to automate the removal of timesheet codes from non-job-related events \nwithin ServiceTitan’s scheduling and timesheet system. This ensures data cleanliness, accurate reporting, \nand prevents misallocation of payroll or operational hours to incorrect categories.\n\nScope:\nThis script will perform the following:\n\n1. Connect to ServiceTitan API\n   - Authenticate using secure API credentials\n   - Retrieve timesheet data and associated events within a specified date range or rolling period\n\n2. Identify Non-Job Events\n   - Filter out events that are not linked to active jobs, based on criteria such as missing jobId or event type classification\n\n3. Check and Remove Timesheet Codes\n   - For each non-job event, inspect for assigned tim

In [4]:
def Pretty_Json(json_object):
    
    json_formatted_str = json.dumps(json_object, indent=2)

    print(json_formatted_str)

In [5]:
def Login(CLIENT_ID,CLIENT_SECRET):
    print("Logging in...")

    # Integration Endpoint
    # url = "https://auth-integration.servicetitan.io/connect/token"

    # Production Endpoint
    url = "https://auth.servicetitan.io/connect/token"

    payload = f"grant_type=client_credentials&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    response = requests.request("POST", url, data=payload, headers=headers)

    print("Logged in!")

    return(response.text)

In [6]:
# NonJobAppointments_GetList
# Gets a list of non-job appointments
def get_nonjobappointments(ACCESS_TOKEN,TENANT_ID,APP_KEY):
    print(TENANT_ID,APP_KEY)

    url = f"https://api.servicetitan.io/dispatch/v2/tenant/{TENANT_ID}/non-job-appointments"

    # Get current UTC time
    current_time = datetime.utcnow()-timedelta(days=7)

    # Format as RFC3339 (ISO 8601) with microseconds and 'Z' for UTC
    formatted_time = current_time.isoformat(timespec='microseconds') + 'Z'

    print("Current time (RFC 3339):", formatted_time)

    payload = {
        "activeOnly": True,
        "timesheetCodeId": 320,
        "pageSize": 500,
        "createdOnOrAfter":formatted_time
    }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }

    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("GET", url, params=payload, headers=headers)

    TARGET_JSON = json.loads(response.text)
    
    # #DEBUG
    with open('timesheet_data.json', 'w') as f:
        json.dump(TARGET_JSON, f, indent=4)

# print(response.text)
    return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)

In [ ]:
#NonJobAppointments_Update
#Update an existing non-job appointment

def put_eventbyids(id,ACCESS_TOKEN,TENANT_ID,APP_KEY,EVENT_NAME,EVENT_DURATION,EVENT_START,EVENT_TECHID):


    ################################################################################
    ## FIND NONJOBEVENT
    url = f"https://api.servicetitan.io/dispatch/v2/tenant/{TENANT_ID}/non-job-appointments/{id}"
    # payload = {
    #     "technicianId": EVENT_TECHID,
    #     "start": EVENT_START,
    #     "duration": EVENT_DURATION,
    #     "name": EVENT_NAME,
    #     "timesheetCodeId": 0
    # }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }
    # print(id,payload)
    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("GET", url, params={}, headers=headers)

    TARGET_JSON = json.loads(response.text)
    print(TARGET_JSON)

    old_id=TARGET_JSON['id']

    #DEBUG
    with open('BEFORE_EVENT.json', 'w') as f:
        json.dump(TARGET_JSON, f, indent=4)
    ################################################################################
    ## POST NEW EVENT WITH NO TIMECODE
    payload = {
        "technicianId": TARGET_JSON['technicianId'],
        "start": TARGET_JSON['start'],
        "duration": TARGET_JSON['duration'],
        "name": TARGET_JSON['name'],
        "summary": TARGET_JSON['summary'],
        "removeTechnicianFromCapacityPlanning":True
        # "timesheetCodeId": TARGET_JSON['timesheetCodeId']
        # "timesheetCodeId": none
    }
    url = f"https://api.servicetitan.io/dispatch/v2/tenant/{TENANT_ID}/non-job-appointments"
    # payload = {
    #     "technicianId": EVENT_TECHID,
    #     "start": EVENT_START,
    #     "duration": EVENT_DURATION,
    #     "name": EVENT_NAME,
    #     "timesheetCodeId": 0
    # }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }
    # print(id,payload)
    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("POST", url, data=payload, headers=headers)
    TARGET_JSON = json.loads(response.text)
    print(TARGET_JSON)
    #DEBUG
    with open('AFTER_EVENT.json', 'w') as f:
        json.dump(TARGET_JSON, f, indent=4)
    ################################################################################   327933056?appointmentId=327933441
    ## DEL OLD NONJOBEVENT
    url = f"https://api.servicetitan.io/dispatch/v2/tenant/{TENANT_ID}/non-job-appointments/{old_id}"
    
    # payload = {
    #     "technicianId": EVENT_TECHID,
    #     "start": EVENT_START,
    #     "duration": EVENT_DURATION,
    #     "name": EVENT_NAME,
    #     "timesheetCodeId": 0
    # }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }
    # print(id,payload)
    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("DELETE", url, params={}, headers=headers)

    TARGET_JSON = json.loads(response.text)
    print(TARGET_JSON)

    #DEBUG
    with open('BEFORE_EVENT.json', 'w') as f:
        json.dump(TARGET_JSON, f, indent=4)
    ################################################################################

# print(response.text)
    return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)

In [8]:
def patch_invoicebyID(id,items,ACCESS_TOKEN,TENANT_ID,APP_KEY):

    url = f"https://api.servicetitan.io/accounting/v2/tenant/{TENANT_ID}/invoices/{id}/items"
    payload = {
        "items" : items
    }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }

    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("PATCH", url, data=items, headers=headers)

    # TARGET_JSON = json.loads(response.text)
    print(response.text)
    # return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)

In [9]:
# Finding items in target_invoice with items labeled as "(Labor)" add them to labor_items
# and return labor_items
def parse_labor_items(target_invoice,labor_items):

    #Iterate through target_invoice and add them to labor items
    for item in target_invoice['data'][0]['items']:

        #Checks if display name contains "(Labor)"
        if ("(Labor)" in item['displayName']):

            # Append Found Labor item to end of list
            labor_items = np.append(labor_items,item)
    
    # Return all labor items
    return(labor_items)

In [10]:
# Zeros out labor items
def zero_labor_items(labor_items,techID):


    #Int new list
    zeroed_items = np.array([])

    #Iterate through labor_items and add them to new list of items
    # for item in target_invoice['data'][0]['items']:
    for item in labor_items:
        if ("(Labor)" in item['displayName']):

            # Int object
            found_item = {}

            # Create new object to insert into invoice
            found_item['skuId'] = item['skuId']
            found_item['skuName'] = item['skuName']
            found_item['description'] = item['description']
            found_item['quantity'] =  item['quantity']
            found_item['cost'] = 0.00
            found_item['id'] = item['id']
            #This is required to batch. Set currently to TPT Tech Test
            found_item['technicianId'] = techID # Generic
            # found_item['technicianId'] = 71898136 # LEES
            # found_item['technicianId'] = 28994682 # RPATLANTA
            # found_item['technicianId'] = 4589457 # Munz
            # found_item['technicianId'] = 5385807 # RPUtah


            #Add object to list
            zeroed_items = np.append(zeroed_items,found_item)

    return(zeroed_items)

In [11]:
# Main Driver
def remove_codes(CLIENT_KEYS):
    
    token = Login(CLIENT_KEYS['CLIENT_ID'],CLIENT_KEYS['CLIENT_SECRET'])
    json_token = json.loads(token)
    ###
    # DEBUG: Print Token
    # Pretty_Json(json_token)
    ###
    #Parse Access Token
    ACCESS_TOKEN = (json_token['access_token'])

    # Target Invoice (TDB to find by serach function)
    # INVOICE_ID = 30567971
    # Parameter: InvoiceID
    target = get_nonjobappointments(ACCESS_TOKEN,CLIENT_KEYS['TENANT_ID'],CLIENT_KEYS['APP_KEY'])
    print(target)
    # print(target.keys())
    length_of_array = str(len(target['data'])) + " events"
    print(length_of_array)
    
    for item in target['data']:
        # print(item['id'])
        EVENT_ID = item['id']
        EVENT_NAME = item['name']
        EVENT_DURATION = item['duration']
        EVENT_START = item['start']
        EVENT_TECHID = item['technicianId']

        if(("Personal Event" in EVENT_NAME) or ("OOO" in EVENT_NAME)):
            target = put_eventbyids(EVENT_ID,ACCESS_TOKEN,CLIENT_KEYS['TENANT_ID'],CLIENT_KEYS['APP_KEY'],EVENT_NAME,EVENT_DURATION,EVENT_START,EVENT_TECHID)
        # break
        ###
        # DEBUG: Print Invoice and Invoice Keys
        # Pretty_Json(target['data'][0])
        # target['data'][0].keys()
        ###
"""
        target_invoice = target
        #Initialize empty array to find labor
        # Can this see my invoice???
        labor_items = np.array([])
        SamIAM = parse_labor_items(target_invoice,labor_items)
        # print(SamIAM)
        # print(len(SamIAM))
        CHECK = False
        if (len(SamIAM) == 0):
            continue

        for item in SamIAM:
            if(item['totalCost'] != '0.00'):
                # print("Found Item that is not zero", item)
                CHECK = True

        if (CHECK == False):
            continue
        
        
        SamIAM = zero_labor_items(SamIAM, CLIENT_KEYS['techID'])

        print("##### Zeroing out Items:", INVOICE_ID)

        #Invoices_UpdateInvoice
        for item in SamIAM:
            patch_invoicebyID(INVOICE_ID,item,ACCESS_TOKEN,CLIENT_KEYS['TENANT_ID'],CLIENT_KEYS['APP_KEY'])
    """

'\n        target_invoice = target\n        #Initialize empty array to find labor\n        # Can this see my invoice???\n        labor_items = np.array([])\n        SamIAM = parse_labor_items(target_invoice,labor_items)\n        # print(SamIAM)\n        # print(len(SamIAM))\n        CHECK = False\n        if (len(SamIAM) == 0):\n            continue\n\n        for item in SamIAM:\n            if(item[\'totalCost\'] != \'0.00\'):\n                # print("Found Item that is not zero", item)\n                CHECK = True\n\n        if (CHECK == False):\n            continue\n        \n        \n        SamIAM = zero_labor_items(SamIAM, CLIENT_KEYS[\'techID\'])\n\n        print("##### Zeroing out Items:", INVOICE_ID)\n\n        #Invoices_UpdateInvoice\n        for item in SamIAM:\n            patch_invoicebyID(INVOICE_ID,item,ACCESS_TOKEN,CLIENT_KEYS[\'TENANT_ID\'],CLIENT_KEYS[\'APP_KEY\'])\n    '

In [12]:
#### Failed Test Report Beta
# https://api.servicetitan.io/reporting/v2/tenant/{tenant}/report-categories[?page][&pageSize][&includeTotal]
def get_reportCategoryID(ACCESS_TOKEN,TENANT_ID):
    exit()

    url = f"https://api.servicetitan.io/reporting/v2/tenant/{TENANT_ID}/report-categories"
    payload = {
        # "ids": ids
        }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }

    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("GET", url, params=payload, headers=headers)

    TARGET_JSON = json.loads(response.text)
# print(response.text)
    return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)

In [13]:
#### Failed Test Report Beta
#Report ID
# 35798994
#https://api.servicetitan.io/reporting/v2/tenant/{tenant}/report-category/{report_category}/reports/{reportId}
def get_reportbyID(REPORTID,REPORT_CATEGORY,ACCESS_TOKEN):
    exit()

    url = f"https://api.servicetitan.io/reporting/v2/tenant/{TENANT_ID}/report-category/{REPORT_CATEGORY}/reports/{REPORTID}"
    payload = { }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }

    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("GET", url, params=payload, headers=headers)

    TARGET_JSON = json.loads(response.text)
# print(response.text)
    return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)


In [14]:
#### Failed Test Report Beta
#### https://api.servicetitan.io/reporting/v2/tenant/{tenant}/report-category/{report_category}/reports[?page][&pageSize][&includeTotal]
def post_reportbyID(REPORTID,REPORT_CATEGORY,ACCESS_TOKEN):
    exit()

    url = f"https://api.servicetitan.io/reporting/v2/tenant/{tenant}/report-category/{report_category}/reports[?page][&pageSize][&includeTotal]"
    payload = { 
        "page" : 1,
        # "pageSize" : 100,
        # "includeTotal" : True
    }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }

    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("POST", url, params=payload, headers=headers)

    TARGET_JSON = json.loads(response.text)
# print(response.text)
    return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)

In [15]:
#### Failed Test Report Beta
#### https://api.servicetitan.io/reporting/v2/tenant/{tenant}/report-category/{report_category}/reports[?page][&pageSize][&includeTotal]
def get_reports(REPORT_CATEGORY,ACCESS_TOKEN,TENANT_ID):
    exit()

    url =  f"https://api.servicetitan.io/reporting/v2/tenant/{TENANT_ID}/report-category/{REPORT_CATEGORY}/reports"
    # payload = {
    #     "page" : 1
    # }
    headers = {
        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}',
        "Content-Type": "application/json"
    }
    # response = requests.request("GET", url, params=payload, headers=headers)


    # url = "https://api-integration.servicetitan.io/reporting/v2/tenant/%7Btenant%7D/report-category/%7Breport_category%7D/reports"

    querystring = {"page":"1","pageSize":"50","includeTotal":"true"}

    payload = ""
    

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

# print(response.text)

In [16]:
#### Failed Test Report Beta
#### https://api.servicetitan.io/reporting/v2/tenant/{tenant}/report-category/{report_category}/reports/{reportId}/data[?page][&pageSize][&includeTotal]
def post_reportbyID(REPORTID,REPORT_CATEGORY,ACCESS_TOKEN,TENANT_ID):
    exit()

    url = f"https://api.servicetitan.io/reporting/v2/tenant/{TENANT_ID}/report-category/{REPORT_CATEGORY}/reports/{REPORTID}/data"
    payload = { 
        "page" : 1
        # "pageSize" : 100,
        # "includeTotal" : True
    }
    headers = {        "Authorization": f'{ACCESS_TOKEN}', 
        "ST-App-Key": f'{APP_KEY}'
    }

    # response = requests.request("GET", url, data=payload, headers=headers)
    response = requests.request("POST", url, params=payload, headers=headers)

    TARGET_JSON = json.loads(response.text)
# print(response.text)
    return(TARGET_JSON)
# Pretty_Json(TARGET_JSON)

In [17]:
# #### Driver Testing --> FAILED due to BETA TEST
# token = Login(Munz["CLIENT_ID"],Munz["CLIENT_SECRET"])
# json_sample = json.loads(token)
# ###
# # DEBUG: Print Token
# # Pretty_Json(json_sample)
# ###
# #Parse Access Token
# ACCESS_TOKEN = (json_sample['access_token'])
# test = get_reportCategoryID(ACCESS_TOKEN, Munz['TENANT_ID'])
# Pretty_Json(test)

# test3 = get_reports("other",ACCESS_TOKEN,Munz['TENANT_ID'])

# Pretty_Json(test3)

# test2 = post_reportbyID("35798994","Other",ACCESS_TOKEN,Munz["TENANT_ID"])

# # Pretty_Json(test2)

In [18]:
import time

# Dictionary to store active timers
timers = {}

def start_timer(name):
    if name in timers:
        print(f"Timer '{name}' is already running.")
    else:
        timers[name] = time.time()
        print(f"Started timer '{name}'.")

def end_timer(name):
    if name not in timers:
        print(f"Timer '{name}' not found or already ended.")
    else:
        start_time = timers.pop(name)
        elapsed_time = time.time() - start_time
        print(f"Ended timer '{name}'. Duration: {elapsed_time:.2f} seconds.")

def get_timer(name):
    if name not in timers:
        print(f"Timer '{name}' not found.")
    else:
        elapsed_time = time.time() - timers[name]
        print(f"Timer '{name}' is running. Elapsed time: {elapsed_time:.2f} seconds.")

In [19]:
#Main driver
def main():

    #Load and assign API Keys to Variable
    load_dotenv()

    Keys = os.getenv("KEYS")
    print("Current Tenant List: " + str(Keys))
    CLIENT_KEYS = np.array([])
    for KEY in json.loads(Keys):
        # print(KEY)
        client_key = json.loads(os.getenv(f"{KEY}"))
        client_key['TENANT_NAME'] = KEY
        # print(test)
        # Munz = json.loads(test)
        CLIENT_KEYS = np.append(CLIENT_KEYS, client_key)

    # print(len(CLIENT_KEYS))
    #  for task_name, task_duration in tasks:
    #     thread = threading.Thread(target=example_task, args=(task_name, task_duration))
    #     threads.append(thread)
    #     thread.start()

    threads = []

    for CLIENT_KEY in CLIENT_KEYS:
        
        current_time = datetime.now()

        # Print the current time
        print("Current time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))


        start_timer(CLIENT_KEY['TENANT_NAME'])
        
        print("#####",CLIENT_KEY['TENANT_NAME'],"#####")

        # thread = threading.Thread(target=zero_material, args=(CLIENT_KEY))
        # threads.append(thread)
        # thread.start()
        
        #Driving Main function
        remove_codes(CLIENT_KEY)

        end_timer(CLIENT_KEY['TENANT_NAME'])
        # get_timer(CLIENT_KEY['TENANT_NAME']) # Error on get_timer afer end_timer

    # Wait for all threads to complete
    # for thread in threads:
    #     thread.join()
    # 
    # 
    #     
   #######################################
    # with open("InvoiceIds.txt","r") as IDs:
    #     text = IDs.readlines()
    #     new_text = np.array([])
    #     for item in text:
    #         item = item.strip()
    #         new_text = np.append(new_text,item)
    #     print(new_text)
    #######################################

In [20]:
if (__name__ == "__main__"):
    while True:
        current_time = datetime.now()
        # Print the current time
        print("Current time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))
        main()
        current_time = datetime.now()
        # Print the current time
        print("Current time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))
        break
        time.sleep(15 * 60)  # 15 minutes * 60 seconds


Current time: 2025-05-12 15:47:31
Current Tenant List: ["SUMMIT"]
Current time: 2025-05-12 15:47:31
Started timer 'SUMMIT'.
##### SUMMIT #####
Logging in...
Logged in!
3942606357 ak1.c620nhtxdcsw0dvyo5vus3ab4
Current time (RFC 3339): 2025-05-05T22:47:31.880477Z


C:\Users\Sando\AppData\Local\Temp\ipykernel_42712\2841402409.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow()-timedelta(days=7)


{'page': 1, 'pageSize': 500, 'hasMore': False, 'totalCount': None, 'data': [{'id': 327866379, 'technicianId': 1862806, 'start': '2025-05-09T14:00:00Z', 'name': 'Deliver estimates possibly sign', 'duration': '03:00:00', 'timesheetCodeId': 320, 'summary': None, 'clearDispatchBoard': False, 'clearTechnicianView': False, 'removeTechnicianFromCapacityPlanning': True, 'allDay': False, 'showOnTechnicianSchedule': False, 'active': True, 'createdOn': '2025-05-08T21:30:16.3816332Z', 'modifiedOn': '2025-05-08T21:30:16.4048241Z', 'createdById': 1857239}, {'id': 327887638, 'technicianId': 1862662, 'start': '2025-05-09T23:30:00Z', 'name': 'Call Linda Chapman', 'duration': '01:00:00', 'timesheetCodeId': 320, 'summary': None, 'clearDispatchBoard': False, 'clearTechnicianView': False, 'removeTechnicianFromCapacityPlanning': True, 'allDay': False, 'showOnTechnicianSchedule': False, 'active': True, 'createdOn': '2025-05-09T20:38:36.2135405Z', 'modifiedOn': '2025-05-09T20:38:36.2269945Z', 'createdById': 1

In [21]:
#LEES ERROR
# {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.8","title":"Invoice item is not found","status":409,"traceId":"8cc542a7cf6c226f-MIA","errors":{"":["
# 
# Invoice item is not found"]},"data":{}}
# ['23858873' '5739431' '5739677' '32969373' '32969367' '53931841'
#  '55876858' '56821975']

#PLANET ROOF NOT RUN

#RPATLANTA
# {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.8","title":"SQL error occurred.\nStorage error details \u0027Entity: UsedMaterial; Field: Truck.Id (FieldInfo);\u0027\nSQL error details \u0027Type: CheckConstraintViolation; Table: UsedMaterial; Column: Truck.Id;\u0027\nQuery \u0027UPDATE [roofingpupsatlanta].[UsedMaterial]  SET [Technician.Id] = @p0_0, [Truck.Id] = @p0_1, [ModifiedOn] = @p0_2 WHERE  ([UsedMaterial].[Id] = @p0_3); [p0_0=\u00274589457\u0027;p0_1=\u0027\u0027;p0_2=\u00272024-10-02 14:11:51.3997026\u0027;p0_3=\u002748220798\u0027]\u0027\nOriginal message \u0027Cannot insert the value NULL into column \u0027Truck.Id\u0027, table \u0027Prod-201904ex.roofingpupsatlanta.UsedMaterial\u0027; column does not allow nulls. UPDATE fails.\nThe statement has been terminated.\u0027","status":409,"traceId":"8cc548b13bbe5f1f-MIA","errors":{"":["Cannot insert the value NULL into column \u0027Truck.Id\u0027, table \u0027Prod-201904ex.roofingpupsatlanta.UsedMaterial\u0027; column does not allow nulls. UPDATE fails.\nThe statement has been terminated."]},"data":{}}
# ['47612935']

# RPMARYLAND
# NOT RUN

#RPUTAH ERROR -> FIXED
# {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.8","title":"Invoice item is not found","status":409,"traceId":"8cc54f07de3f8dc0-MIA","errors":{"":["Invoice item is not found"]},"data":{}}
# ['26175877']



In [22]:
# !python -m ssl
# !pip install libssl-dev



In [23]:
# import threading
# import time

# # Example task function
# def example_task(name, duration):
#     print(f"Task {name} started.")
#     time.sleep(duration)
#     print(f"Task {name} completed in {duration} seconds.")

# # Function to run multiple threads
# def run_multithreading():
#     # Create threads for different tasks
#     threads = []
#     tasks = [("Task1", 2), ("Task2", 3), ("Task3", 1)]
    
#     for task_name, task_duration in tasks:
#         thread = threading.Thread(target=example_task, args=(task_name, task_duration))
#         threads.append(thread)
#         thread.start()

#     # Wait for all threads to complete
#     for thread in threads:
#         thread.join()

# # Run the function to see multithreading in action
# if __name__ == "__main__":
#     start_time = time.time()
#     run_multithreading()
#     end_time = time.time()
#     print(f"All tasks completed in {end_time - start_time:.2f} seconds.")
